In [1]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from collections import defaultdict

df = pd.read_csv('data_t1.csv', nrows=1_000)

df['create_date'] = pd.to_datetime(df['create_date'])
df['update_date'] = pd.to_datetime(df['update_date'])

df.fillna('Не указано', inplace=True)

def calculate_similarity(str1, str2):
    return fuzz.ratio(str1, str2)

def compute_similarities(df, columns, threshold=80):
    similar_records = defaultdict(list)

    for i in range(len(df)):
        for j in range(i + 1, len(df)):
            similarity_score = 0

            for col in columns:
                similarity_score += calculate_similarity(str(df.loc[i, col]), str(df.loc[j, col]))

            if similarity_score / len(columns) >= threshold:
                similar_records[i].append(j)
                similar_records[j].append(i)
    
    return similar_records

columns = ['client_first_name', 'client_middle_name', 'client_last_name', 'client_fio_full', 'client_bday', 'client_bplace', 'client_cityzen', 'client_resident_cd', 'client_gender']

similar_records = compute_similarities(df, columns, threshold=80)

def get_gold_record(indices):
    records = df.iloc[indices]
    gold_record = records.loc[records['update_date'].idxmax()]
    return gold_record

final_output = []

for i in range(len(df)):
    similar_indices = similar_records.get(i, [])
 
    gold_record = get_gold_record([i] + similar_indices)
  
    similar_count = len(similar_indices) + 1  

    final_output.append({
        'client_fio_full': gold_record['client_fio_full'],
        'similar_records': similar_count,
        'total_sim_cnt': 1  
    })

output_df = pd.DataFrame(final_output)

print(output_df[['client_fio_full', 'similar_records', 'total_sim_cnt']].to_string(index=False))

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
/var/folders/28/ry94lb9x7g9dlmnxp72zr5rr0000gn/T/ipykernel_16649/581915057.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Не указано' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('Не указано', inplace=True)


                         client_fio_full  similar_records  total_sim_cnt
                Евдунов Меджид Кузарович                1              1
                 Повод Мансыр Хирсанович                1              1
              БУРНАС НУРДИНБЕК УВАЙСОВИЧ                1              1
            Бондажевская Афаят Гордиевна                1              1
                  Разуева Сахия Фидаевна                1              1
                Чередник Аржана Наилович                1              1
                Богач Аимкан Агургеновна                1              1
                       Помилов Арли Янис                1              1
                    Лыло Магарам Тайевич                1              1
                  Лехно Тойдык Чукановна                1              1
             Причина Курбонхон Жумакович                1              1
                Драницин Эвальд Сайдович                1              1
               Чихирев Эрдни Дальсебович           